In [1]:
import os
import cv2
import numpy as np
import torch
import dlib
from imutils import face_utils
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from torchvision import transforms  
import torch.nn as nn
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.nn.functional as F
import math
import time
from imutils import face_utils
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import mediapipe
import sys
sys.path.append("../LuminEye-Iris-Center-Localization/")
from BaseModels.resnetModels import BB_model
from BaseModels.efficientnetModels import BB_model
import pandas as pd
from scipy.spatial import distance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


np.random.seed(42)

In [2]:
# BioID
val_csv_path = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/BIOID/BioIDCentersval.csv"


val_df = pd.read_csv(val_csv_path)

In [3]:
def selectImgDir(datasetType:str):
    
    
    
    
    
    if datasetType == "MP2GAZE":
        return "/home/nipun/Documents/Uni_Malta/Datasets/"
        
        
        
    elif datasetType =="i2head":
        return "/home/nipun/Documents/Uni_Malta/Datasets/"
        
    elif datasetType == "GI4E":
        
        return "/home/nipun/Documents/Uni_Malta/Datasets/gi4e_database/images/"

In [4]:
detector = None
predictor = None
GAN_MODEL = None
IRIS_MODEL = None
EYE_AR_THRESH = 0.2

mp_face_mesh = mediapipe.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

In [5]:

#BIOID
padded_amt = 15
def cropped_image(img, shape_array, padded_amt=padded_amt ):
    """Cropped eye region 

    Args:
        img (__numpy__): _Original Image_
        shape_array (_numpy_): _FaceLandMark locations_
        padded_amt (int, optional): _padding size_. Defaults to 15.

    """

    Leye = {"top_left": shape_array[70], "bottom_right": shape_array[133]}

    Reye = {"top_left": shape_array[285],
            "bottom_right": shape_array[263]}

    left_eye = img[Leye["top_left"][1]:Leye["bottom_right"][1] +
                   padded_amt, Leye["top_left"][0]:Leye["bottom_right"][0]]

    right_eye = img[Reye["top_left"][1]:Reye["bottom_right"][1] +
                    padded_amt, Reye["top_left"][0]:Reye["bottom_right"][0]]

    return left_eye, right_eye, Leye, Reye

In [6]:

def rescale_coordinate(coord,original_image,resize_amt):
    
    h,w = original_image.shape[:2]
    coord[0] = int((coord[0]/resize_amt) * w)
    coord[1] = int((coord[1]/resize_amt) * h)
    
    return coord

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
def mpArrayToNumpy(landmark_array, img):

    shape_arr = []

    for landmark in landmark_array.landmark:
        x = landmark.x
        y = landmark.y

        relative_x = int(x * img.shape[1])
        relative_y = int(y * img.shape[0])

        shape_arr.append([relative_x, relative_y])

    return np.array(shape_arr)

In [8]:
def scaleCoorinatesToOriginalImage(pred_coords,eye_margin):
    
    # {'top_left': array([385, 214]), 'bottom_right': array([426, 226])}
    
    x1 = eye_margin["top_left"][0] + pred_coords[0]
    y1 = eye_margin["top_left"][1] + pred_coords[1]
    
    return [x1,y1]

In [9]:
def calculateEuclideanDistance(coord1,coord2):
    return (((coord1[0])-float(coord2[0]))^2  + (float(coord1[0])-float(coord2[0]))^2) ^ 0.5

In [10]:
val_df.head()

,Unnamed: 0,SourceImage,Image_Name,X1,Y1
0,0,BioID_0002,BioID_0002_left.png,0.605263,0.461538
1,1,BioID_0002,BioID_0002_right.png,0.636364,0.500000
2,2,BioID_0010,BioID_0010_left.png,0.604167,0.485714
3,3,BioID_0010,BioID_0010_right.png,0.585366,0.423077
4,4,BioID_0018,BioID_0018_left.png,0.634615,0.482759


In [11]:
img_names = list(val_df["SourceImage"])

text_dir_path = "/home/nipun/Documents/Uni_Malta/Datasets/BioID-FaceDatabase-V1.2/labels/"

In [12]:
#BioId
IMG_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/BioID-FaceDatabase-V1.2/images/"

In [13]:
# histogramPath

histogramPath = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-MainPipeLine/BayesClassifier/MatLab/ScleraSkinIris.txt"

In [14]:
def BayesEstimation(img,histogramBinValuesPath,thresh=2.5):
    
    r_img = img[:,:,::-1][:,:,0]
    
    imgGauss = cv2.GaussianBlur(r_img,(3,3),1)
    
    with open(histogramBinValuesPath,"r") as f:
        hist = [float(j.rstrip()) for j in f.readlines()]
        
    imgR_bin = np.zeros((r_img.shape[0],r_img.shape[1])).astype(np.uint8)
    
    
    
    for row in range(r_img.shape[0]):
    
        for column in range(r_img.shape[1]):
            
            binNo = (imgGauss[row,column]) // 4
            
            
            # print(binNo)
            likelihood = hist[binNo]
            
            
            if likelihood >= thresh:
                imgR_bin[row,column] = 1
                
                
    # plt.imshow(imgR_bin)
    # plt.show()
    
    
    
    analysis = cv2.connectedComponentsWithStats(imgR_bin,
                                           4,
                                            cv2.CV_32S)
    (totalLabels, label_ids, values, centroid) = analysis
    
    
    return centroid[-1]

In [15]:
def scaleCoorinatesToOriginalImage(pred_coords,eye_margin):
    
    # {'top_left': array([385, 214]), 'bottom_right': array([426, 226])}
    
    x1 = eye_margin["top_left"][0] + pred_coords[0]
    y1 = eye_margin["top_left"][1] + pred_coords[1]
    
    return [x1,y1]

In [16]:
maximizedNormalizedError = []

count = 0
for name in img_names:
    
    img_path = os.path.join(IMG_DIR,name+".pgm")
    
    text_path = os.path.join(text_dir_path,name+".eye")
    
    
    with open(text_path,"r") as f:
        
        data = f.readlines()[-1].rstrip().split("\t")
    
    
    right_center = [float(num)  for num in data[:2]]
    left_center = [float(num) for num in data[2:]]
    
    
    
   

    
    IPD = distance.euclidean(left_center, right_center)
    
    
    img = cv2.imread(img_path)

    results = face_mesh.process(img)

    if results.multi_face_landmarks is not None:
        landmarks = results.multi_face_landmarks[0]

        shape_arr = mpArrayToNumpy(landmarks, img)
        
        
        cv2.circle(img, (int(left_center[0]), int(
            left_center[1])), 1, (255, 0, 0), -1)
      

        
        cv2.circle(img, (int(right_center[0]), int(
            right_center[1])), 1, (255, 0, 0), -1)

        left_eye, right_eye, Leye, Reye = cropped_image(img, shape_arr)
        
        
        
        
        
        left_centroid = BayesEstimation(left_eye,histogramPath)
        right_centroid = BayesEstimation(right_eye,histogramPath)
        
        
        
        # Cropped Left Ey Predition
        cv2.circle(left_eye, (int(left_centroid[0]), int(
            left_centroid[1])), 1, (0, 255, 0), -1)
        
        # Cropped Right Ey Predition
        cv2.circle(right_eye, (int(right_centroid [0]), int(
            right_centroid [1])), 1, (0, 255, 0), -1)
        
        
        
        pred_l_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            left_centroid , Leye)
        pred_r_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            right_centroid, Reye)
        
        
        
        cv2.circle(img, (int(pred_l_eye_toOriginaImage[0]), int(
            pred_l_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        cv2.circle(img, (int(pred_r_eye_toOriginaImage[0]), int(
            pred_r_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)
        
        
        
        
        
        
        
        left_eye_euclidea_distance = distance.euclidean(
            pred_l_eye_toOriginaImage, left_center)

        right_eye_euclidea_distance = distance.euclidean(
            pred_r_eye_toOriginaImage, right_center)
        
        
        eMax = max(left_eye_euclidea_distance, right_eye_euclidea_distance)/IPD

        print(eMax)

        maximizedNormalizedError.append(eMax)
        
        
        
        # fig,axes = plt.subplots(1,3)
        
        # fig.set_figwidth(15)
        # fig.set_figheight(15)
        
    
        # axes[0].imshow(img)
        # axes[1].imshow(left_eye)
        # axes[2].imshow(right_eye)
        
        
        # axes[0].axis("off")
        # axes[1].axis("off")
        # axes[2].axis("off")

        # plt.tight_layout()
        # plt.show()
        
        
      
        count+=1
        

        # if count==100:
        #     break

0.040225052221847114
0.040225052221847114
0.031724140444672366
0.031724140444672366
0.006977687690572504
0.006977687690572504
0.007461855486748755
0.007461855486748755
0.01643456946396406


0.01643456946396406
0.005739453936449063
0.005739453936449063
0.0050475446512504
0.0050475446512504
0.0113127029700755
0.0113127029700755
0.006399615010741264
0.006399615010741264
0.017411723255185425
0.017411723255185425
0.006405520350200914
0.006405520350200914
0.00628930817610054
0.00628930817610054
0.02204888936844999
0.02204888936844999
0.030926922120984954
0.030926922120984954
0.01608566545176449
0.01608566545176449
0.00271002710027108
0.00271002710027108
0.019596461424998716
0.019596461424998716
0.0058591989449844105
0.0058591989449844105
0.06809825741021604
0.06809825741021604
0.001465626688219714
0.001465626688219714
0.0024277673637832873
0.0024277673637832873
0.002429697854636751
0.002429697854636751
0.0
0.0
0.0
0.0
0.30765919650872975
0.30765919650872975
0.3064868842872537
0.3064868842872537
0.32619386723908567
0.32619386723908567
0.09770001847951752
0.09770001847951752
0.1859274983576661
0.1859274983576661
0.06635705385056036
0.06635705385056036
0.0826431572469399
0.0826431

In [17]:
def CheckForLess(list1, val):
    
    l1 = []
    for x in list1:
        
        if x <= val:
            l1.append(x)
    return l1

In [18]:
e_0_25 = CheckForLess(maximizedNormalizedError,0.25)

e_0_05 = CheckForLess(maximizedNormalizedError,0.05)

e_0_1 = CheckForLess(maximizedNormalizedError,0.1)

In [19]:
(len(e_0_25)/len(maximizedNormalizedError)) *100

85.71428571428571

In [20]:
(len(e_0_05)/len(maximizedNormalizedError)) * 100

35.27696793002916

In [21]:
len(e_0_1)/len((maximizedNormalizedError)) * 100

57.7259475218659